#Run this notebook to construct the feature matrix from aggregate protein-, pair-level data sources.
      Estimated running time ~18h

No required inputs, however, if data retrieved previously, default is not to overwrite

Outputs:
1.   All possible CORUM-2018 labeled pairs:

  *allPairs_corumLabeled.tsv.tar.bz2*

2.   All possible pairs from all known protein-coding genes:

  *allPairs_proteinCoding_geneidFmt.tsv.tar.bz2*

3.   Labeled pairs (for training), detailed:

  *trainingPairs_sourcesLabeled_detailed_corum2018_humap1+2_lm2019.tsv.tar.bz2*

4.   Labeled pairs (for training), simple:

  *trainingPairs_labeledAggregate_corum2018_humap1+2_lm2019.tsv.tar.bz2*
    
5.   All possible, unlabeled pairs from available protein-coding genes:

  *nontrainingPairs_proteinCoding.tsv.tar.bz2*
    
6.   hu.MAP 1.0 training features:

  *featMat_trainingPairs_humap1.tsv.tar.bz2*
    
7.   hu.MAP 1.0 (unlabeled pairs) features:

  *featMat_unlabeledPairs_humap1.tsv.tar.bz2*

8.   hu.MAP 2.0 training features:

  *featMat_trainingPairs_humap2.tsv.tar.bz2*

9.   hu.MAP 2.0 (unlabeled pairs) features:

  *featMat_unlabeledPairs_humap2.tsv.tar.bz2*
    
10.   BioPlex 3.0 training features:

  *featMat_trainingPairs_bioplex3.tsv.tar.bz2*  

11.   BioPlex 3.0 (unlabeled pairs) features:

  *featMat_unlabeledPairs_bioplex3.tsv.tar.bz2*

12.   Aggregate pair-level training features:

  *featMats_aggLabeled.tsv.tar.bz2*     

13.   Aggregate pair-level (unlabeled pairs) features:

  *featMats_aggUnlabeled.tsv.tar.bz2*   




#Install packages and load libraries

Import common packages

In [ ]:
%%capture
import argparse, cProfile, datetime
from functools import reduce
import glob
import itertools as it
from multiprocessing import cpu_count, Pool
import networkx as nx
import numpy as np
import os

import pandas as pd
import pickle
import matplotlib.pyplot as plt
import random, re, shutil
from scipy.stats import hmean
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.metrics import auc, average_precision_score, precision_recall_curve
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import subprocess as sp
import sys
import tempfile as tf
import time
from tqdm import tqdm as progressMonitor

Enable Google Colab, mount drives, and load proprietary modules...

In [ ]:
%%capture
from google.colab import drive, files, output
drive.mount('/content/drive', force_remount=True)
!rm -r sample_data/

In [ ]:
#useful to keep track of sys vars so as to better monitor space remaining
print(cpu_count())
sysVars = list(globals().keys())

2


In [ ]:
rootDir = '/content/drive/My Drive/'
workDir = rootDir + 'elcfs_protein_complex_modeling/'  #phase 2 directory
workDir_elcfs = rootDir + 'Primary Research/proteinPairs_complexMaps/' #modeling library
workDir_ph1 = rootDir + 'Primary Research/JLMwSCBC_notebook/'  #phase 1 directory

sys.path.insert(0, rootDir)
for p in workDir, workDir_elcfs, workDir_ph1: sys.path.append(p)

In [ ]:
from util import ppiPrediction_v2, dataProcessing, modelEvaluating
from utils import operations, reference, alertMe
pushoverKey_user = 'uith8rmy2npjj1oqpjwcanow3un984'
pushoverAPI = 'aw4v3424kaznrw598r6qge9icddwg7'

In [ ]:
#copy the contents between the ellipses into the cell subsequent to that for
# which the run time is long
'''
#put this block after any cell expected to take a long time
cmdReport = alertMe.statusCheck(pushoverAPI, pushoverKey_user)
cmdReport.finishPush()
'''

'\n#put this block after any cell expected to take a long time\ncmdReport = alertMe.statusCheck(pushoverAPI, pushoverKey_user)\ncmdReport.finishPush()\n'

#Sources

1.   [CORUM](https://mips.helmholtz-muenchen.de/corum/#download)

2.   [hu.MAP 1.0](http://hu.proteincomplexes.org/download) (HEK293T)
>    *(Obsolete)*
> *   BioPlex v1 - Huttlin 2015 (AP-MS)
> *   Hein 2015 (AP-MS)
> *   Wan 2015 (CF-MS)

3.   [hu.MAP 2.0](http://humap2.proteincomplexes.org/download) (HEK293T)
> *  BioPlex v1, v2 - Huttlin 2015, 2017 (AP-MS)
> *  Boldt 2016 (AP-MS)
> *  Gupta 2015 (Proximity)
> *  Hein 2015 (AP-MS)
> *  Wan 2015 (AP-MS)
> *  Youn 2018 (Proximity)
> *  Treiber 2017; Mallam 2019 (RNA-Pulldown)

4.   [BioPlex](https://bioplex.hms.harvard.edu/data/BioPlex_BaitPreyPairs_noFilters_293T_10K_Dec_2019.tsv) v3 - Huttlin 2021 (AP-MS)

5.   [Lugo-Martinez 2019](https://drive.google.com/drive/folders/191Y14LBVZnIxWJysmJ7I_UL451aStVvj?usp=sharing)
> *  HPA - Uniprot 2017; Ouyang 2019 (Microscopy; deep-learning features)
> *  NCI-60 - Gholami 2013 (RNA and protein expression)
> *  FANTOM5 - Forrest 2014 (RNA expression)
>> FANTOM6 newest
> *  Uhlén 2015 (RNA expression)
> *  GTEx (v7) - Yizhak 2019 (RNA expression)
>> GTEx (v8) newest

6.   [SubCellBarCode](https://www.subcellbarcode.org/) - Orre 2019 (AP-MS, subcellular localization)

#Generate all labeled pairs from CORUM complexes

*   Co-complex protein pairs: 1

*   Remaining pairs: 0

In [ ]:
#Functionality for fresh download of latest repository unavailable due to
# continuing recovery of website from cyberattack.
# Refer to https://www.biostars.org/p/9563995/
# Downloaded latest repository of CORUM complexes, release 2018.07.01.
corumData = workDir + 'srcData/CORUM/corum.xlsx'
accessCORUM = reference.CORUM(corumData)
humanProts = set(accessCORUM.humanProts)
print(len(humanProts))

3375


In [ ]:
allPairs_corumLabeled = \
  pd.DataFrame(accessCORUM.allPairs_human,
               columns=['id1', 'id2'], dtype='str')
allPairs_corumLabeled = \
  operations.freezePairs(allPairs_corumLabeled, 'id1', 'id2')
allPairs_corumLabeled.insert(3, 'label', 0)

posPairs_humanProt = \
  operations.flatten(
      [[frozenset(pair) for pair in list(it.combinations(cplx, 2))]
       for cplx in accessCORUM.humanCplx_frozen])
allPairs_corumLabeled.loc[
    (allPairs_corumLabeled.pairsFrozen.isin(posPairs_humanProt)), 'label'] = 1

print('+/- Pairs DF dims: {0}'.format(allPairs_corumLabeled.shape))
print('+ Pairs DF dims: {0}'.format(
    allPairs_corumLabeled.loc[allPairs_corumLabeled.label==1, :].shape))
print('- Pairs DF dims: {0}'.format(
    allPairs_corumLabeled.loc[allPairs_corumLabeled.label==0, :].shape))

pd.concat(
    [pd.DataFrame(allPairs_corumLabeled.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     allPairs_corumLabeled],
    axis=1).to_csv(workDir + 'setup/allPairs_corumLabeled.tsv.tar.bz2',
                   sep='\t', index=False)

#Generate all possible pairs from protein-coding genes

In [ ]:
entrezGene_data = workDir + \
  'srcData/ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz'
geneSpace = \
  reference.geneSpace(entrezGene_data, workDir=workDir)
geneSpace.make()

In [ ]:
geneIDs_proteinCoding = \
  operations.generatePairs(geneSpace.geneIDs_proteinCoding)
pd.DataFrame(geneIDs_proteinCoding,
             columns=['id1', 'id2'], dtype='str').to_csv(
                 workDir + 'setup/allPairs_proteinCoding_geneidFmt.tsv.tar.bz2',
                 sep='\t', index=False)

#Get data

##*hu.MAP 1.0*

*   2,524,031 (2,523,842 unique) protein-coding GeneIDs pairs
*   2,557,860 (2,557,669 unique) total IDs
*   33,829 (33,827 unique) unmapped IDs

In [ ]:
humap1URL = 'http://hu.proteincomplexes.org/download/'

In [ ]:
humap1Filenames = \
  [humap1URL[:-1].split('/download')[0] + f
   for f in operations.findDownloadables(humap1URL[:-1], ['ppi', 'feat', 'pairsWProb', 'clusters'])]
print(humap1Filenames)

In [ ]:
for f in humap1Filenames:
  operations.download(workDir, 'huMAP1', f, overwrite=True)

##*hu.MAP 2.0*
*   17,268,070 (17,268,060 unique) protein-coding GeneIDs pairs
*   17,564,755 (17,564,745 unique) total IDs
*   296,685 unmapped IDs

In [ ]:
humap2URL = 'http://humap2.proteincomplexes.org/download/'

In [ ]:
humap2Filenames = \
  [humap2URL[:-1].split('/download')[0] + f
   for f in operations.findDownloadables(humap2URL[:-1], ['ppi', 'feat', 'complex'])]
print(humap2Filenames)

In [ ]:
for f in humap2Filenames:
  operations.download(workDir, 'huMAP2', f, overwrite=True)

##*BioPlex 3.0*
*   5,557,133 unique protein-coding GeneIDs pairs
*   5,851,900 (5,704,386 unique) total IDs
*   698 unmapped IDs

In [ ]:
bioplex3URL = 'https://bioplex.hms.harvard.edu/data/'

In [ ]:
bioplex3Filenames = \
  [bioplex3URL[:-1] + f
   for f in operations.findDownloadables(bioplex3URL[:-1])]
print(bioplex3Filenames)

['https://bioplex.hms.harvard.edu/data293T_HCT116_ProteomeComparison.tsv', 'https://bioplex.hms.harvard.edu/data293T_HCT116_ProteomeComparison_columns.txt', 'https://bioplex.hms.harvard.edu/dataBaitPreyPairs_noFilters_BP2a.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_1.0_293T_DirectedEdges.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_1p0_293T_baitList.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_2.0_293T_DirectedEdges.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_2.3_interactionList.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_2p0_293T_baitList.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_3.0_293T_DirectedEdges.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_3.0_HCT116_DirectedEdges.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_3p0_293T_baitList.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_3p0_HCT116_baitList.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_293T_Network_10K_Dec_2019.tsv', 'https://bioplex.hms.harvard.edu/dataBioPlex_Ba

In [ ]:
bioplex3Feat_filename = \
  bioplex3URL + 'BioPlex_BaitPreyPairs_noFilters_293T_10K_Dec_2019.tsv'
operations.download(workDir, 'BioPlex3', bioplex3Feat_filename, overwrite=True)

/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/BioPlex3/BioPlex_BaitPreyPairs_noFilters_293T_10K_Dec_2019.tsv


'/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/BioPlex3/BioPlex_BaitPreyPairs_noFilters_293T_10K_Dec_2019.tsv'

##Protein Atlas hosted downloadable data

In [ ]:
proteinAtlas_url = 'https://www.proteinatlas.org/about/download/'
proteinAtlas_uhlenURL = 'https://www.proteinatlas.org/about/publicationdata/'

In [ ]:
proteinAtlas_filenames = \
  [proteinAtlas_url[:-1] + f
   for f in operations.findDownloadables(proteinAtlas_url[:-1])]
print(proteinAtlas_filenames)

['https://www.proteinatlas.org/about/download/ENSG00000134057.xml', 'https://www.proteinatlas.org/about/download/ENSG00000134057.trig', 'https://www.proteinatlas.org/about/download/ENSG00000134057.tsv', 'https://www.proteinatlas.org/about/download/ENSG00000134057.json', 'https://www.proteinatlas.org/about/downloadhttp://v13.proteinatlas.org', 'https://www.proteinatlas.org/about/download/download/normal_tissue.tsv.zip', 'https://www.proteinatlas.org/about/download/download/pathology.tsv.zip', 'https://www.proteinatlas.org/about/download/download/subcellular_location.tsv.zip', 'https://www.proteinatlas.org/about/download/download/rna_tissue_consensus.tsv.zip', 'https://www.proteinatlas.org/about/download/download/rna_tissue_hpa.tsv.zip', 'https://www.proteinatlas.org/about/download/download/rna_tissue_hpa_description.tsv.zip', 'https://www.proteinatlas.org/about/download/download/rna_brain_hpa.tsv.zip', 'https://www.proteinatlas.org/about/download/download/rna_pfc_brain_hpa.tsv.zip', 'ht

In [ ]:
proteinAtlas_uhlenFilenames = \
  [proteinAtlas_uhlenURL[:-1] + f
   for f in operations.findDownloadables(proteinAtlas_uhlenURL[:-1])]
print(proteinAtlas_uhlenFilenames)

['https://www.proteinatlas.org/about/publicationdata/download/antibodyresponse.csv.zip', 'https://www.proteinatlas.org/about/publicationdata/download/expression_data.TMA.csv.zip', 'https://www.proteinatlas.org/about/publicationdata/download/expression_data.CMA.csv.zip', 'https://www.proteinatlas.org/about/publicationdata/download/expression_data.IF.csv.zip', 'https://www.proteinatlas.org/about/publicationdata/download/rna_transcript.tsv.gz', 'https://www.proteinatlas.org/about/publicationdata/download/scapis_wellness_correlation_network_all_data.txt.gz', 'https://www.proteinatlas.org/about/publicationdatahttps://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE146773', 'https://www.proteinatlas.org/about/publicationdata/download/pubmed33627808_FUCCI-FACS-data.zip', 'https://www.proteinatlas.org/about/publicationdatamailto:contact@proteinatlas.org']


###*FANTOM5*


In [ ]:
[f for f in proteinAtlas_filenames if 'fantom' in f]

['https://www.proteinatlas.org/about/download/download/rna_tissue_fantom.tsv.zip',
 'https://www.proteinatlas.org/about/download/download/rna_brain_fantom.tsv.zip']

In [ ]:
fantom5_filename = \
  'https://www.proteinatlas.org/about/download/download/rna_tissue_fantom.tsv.zip'
operations.download(workDir, 'FANTOM5', fantom5_filename, overwrite=True)

/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/FANTOM5/rna_tissue_fantom.tsv.zip


'/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/FANTOM5/rna_tissue_fantom.tsv.zip'

###*GTEx*


In [ ]:
[f for f in proteinAtlas_filenames if 'gtex' in f]

['https://www.proteinatlas.org/about/download/download/rna_tissue_gtex.tsv.zip',
 'https://www.proteinatlas.org/about/download/download/rna_brain_gtex.tsv.zip',
 'https://www.proteinatlas.org/about/download/download/transcript_rna_gtexretina.tsv.zip']

In [ ]:
gtex_filename = \
  'https://www.proteinatlas.org/about/download/download/rna_tissue_gtex.tsv.zip'
operations.download(workDir, 'gTEX', gtex_filename, overwrite=True)

/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/gTEX/rna_tissue_gtex.tsv.zip


'/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/gTEX/rna_tissue_gtex.tsv.zip'

###*HPA*


In [ ]:
[f for f in proteinAtlas_filenames if 'location' in f]

['https://www.proteinatlas.org/about/download/download/subcellular_location.tsv.zip']

In [ ]:
hpaLocation_filename = \
  'https://www.proteinatlas.org/about/download/download/subcellular_location.tsv.zip'
operations.download(workDir, 'HPA', hpaLocation_filename, overwrite=True)

/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/HPA/subcellular_location.tsv.zip


'/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/HPA/subcellular_location.tsv.zip'

###*Uhlén 2015*


In [ ]:
[f for f in proteinAtlas_uhlenFilenames if 'rna' in f]

['https://www.proteinatlas.org/about/publicationdata/download/rna_transcript.tsv.gz']

In [ ]:
uhlenData_filename = \
  'https://www.proteinatlas.org/about/publicationdata/download/rna_transcript.tsv.gz'
operations.download(workDir, 'Uhlén', uhlenData_filename, overwrite=True)

/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Uhlén/rna_transcript.tsv.gz


'/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Uhlén/rna_transcript.tsv.gz'

##*Hein 2015*


In [ ]:
#Table S2
hein2015_tableS2 = \
  workDir + 'srcData/Hein2015/1-s2.0-S0092867415012702-mmc3.xlsx'
glob.glob(hein2015_tableS2)

['/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Hein2015/1-s2.0-S0092867415012702-mmc3.xlsx']

##*NCI-60*

In [ ]:
#Sourced from Lugo-Martinez directly
glob.glob(workDir + 'srcData/NCI60/*')

['/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/NCI60/nci60_all_expressionPerCellLine.tsv',
 '/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/NCI60/nci60_all_abundanceLFQPerCellLineProteome.tsv',
 '/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/NCI60/nci60_all_abundanceLFQPerTissueDeep.tsv']

##*SubCellBarCode*

subcellbarcode website down as of 1/23/24

In [ ]:
#Table S2
scbc2019_tableS2 = workDir + 'srcData/SubCellBarCode/mmc2.xlsx'
glob.glob(scbc2019_tableS2)

['/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/SubCellBarCode/mmc2.xlsx']

##*STRING*

In [ ]:
#protein pairs


##*Lugo-Martinez 2019 (2020 Update)*

###*protein ID pairs absent from published data (skip this code block)*

In [ ]:
lm2019URL = 'https://www.andrew.cmu.edu/user/jlugomar/protein_complexes.htm'

In [ ]:
'''
lm2019Data = \
  lm2019URL.split('/protein_complexes.htm')[0] + '/' + \
  operations.findDownloadables(lm2019URL)[0]
print(lm2019Data)
'''

"\nlm2019Data =   lm2019URL.split('/protein_complexes.htm')[0] + '/' +   operations.findDownloadables(lm2019URL)[0]\nprint(lm2019Data)\n"

https://www.andrew.cmu.edu/user/jlugomar/docs/code+data.zip


In [ ]:
#operations.download(workDir, 'Lugo-Martinez2019', lm2019Data, overwrite=True)

/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Lugo-Martinez2019/code+data.zip


In [ ]:
'''
lm2019Data = workDir + 'srcData/Lugo-Martinez2019/code+data'
with ZipFile(lm2019Data + '.zip', 'r') as zObject:
    zObject.extractall(path=workDir + 'srcData/Lugo-Martinez2019/')
'''

"\nlm2019Data = workDir + 'srcData/Lugo-Martinez2019/code+data'\nwith ZipFile(lm2019Data + '.zip', 'r') as zObject: \n    zObject.extractall(path=workDir + 'srcData/Lugo-Martinez2019/')\n"

In [ ]:
#glob.glob(lm2019Data + '/*')

['/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Lugo-Martinez2019/code+data/integrated_MM+TSCS+Exp+Abun+Loc_features_all_pairs_model_either_predictions_rf_candidates.txt',
 '/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Lugo-Martinez2019/code+data/integrated_MM+TSCS+Exp+Abun+Loc_features_training_all_pairs_model_either.tsv',
 '/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Lugo-Martinez2019/code+data/integrated_training_labels_all_pairs_model_either.tsv',
 '/content/drive/My Drive/elcfs_protein_complex_modeling/srcData/Lugo-Martinez2019/code+data/getRFresults.m']

In [ ]:
#lm2019Pairs = pd.read_csv(glob.glob(lm2019Data + '/*')[1], sep='\t', header=None, nrows=10)

###*feature, protein ID pairs, and labels only available locally*

**extract protein ID pair data from bz2 tarball using Linux command (python modules are slow)**

In [ ]:
!cd 'drive/My Drive/elcfs_protein_complex_modeling/srcData/' ; tar -jxvf 'Lugo-Martinez2019.tar.bz2' 'Lugo-Martinez2019/integrated_training_all_model.tsv' 'Lugo-Martinez2019/integrated_training_labels_all_model.tsv' 'Lugo-Martinez2019/integrated_test_labels_all_model.tsv' 'Lugo-Martinez2019/integrated_test_all_model.tsv'

Lugo-Martinez2019/integrated_test_labels_all_model.tsv
Lugo-Martinez2019/integrated_training_labels_all_model.tsv
Lugo-Martinez2019/integrated_test_all_model.tsv
Lugo-Martinez2019/integrated_training_all_model.tsv


#Assemble pairs

##Training

After generating all pairs from human proteins in the latest CORUM repo release along with corresponding labels (non-, co-complex pairs), add labeled pairs as included in hu.MAP 1.0, hu.MAP 2.0, and LM2019 (2020 Update).

In [ ]:
humap1Pairs_training = \
  {f: pd.read_csv(f, sep='\t| ', engine='python',
                  dtype='str', names=['id1', 'id2'])
  for f in glob.glob(workDir + 'srcData/huMAP1/*ppis*')}

for f in humap1Pairs_training.keys():
    if 'neg' in f.split('/')[-1]:
      humap1Pairs_training[f].insert(2, 'label', 0)
    else:
      humap1Pairs_training[f].insert(2, 'label', 1)

humap1Pairs_training = \
  pd.concat([df for df in humap1Pairs_training.values()],
            axis=0, ignore_index=True)

humap1Prots = set(humap1Pairs_training.id1.to_list()).union(
    set(humap1Pairs_training.id2.to_list()))

In [ ]:
humap2Pairs_training = \
  {f: pd.read_csv(f, sep='\t| ', engine='python',
                  dtype='str', names=['id1', 'id2'])
  for f in glob.glob(workDir + 'srcData/huMAP2/*ppis*geneid*txt')}

for f in humap2Pairs_training.keys():
    if 'neg' in f.split('/')[-1]:
      humap2Pairs_training[f].insert(2, 'label', 0)
    else:
      humap2Pairs_training[f].insert(2, 'label', 1)

humap2Pairs_training = \
  pd.concat([df for df in humap2Pairs_training.values()],
            axis=0, ignore_index=True)

humap2Prots = set(humap2Pairs_training.id1.to_list()).union(
    set(humap2Pairs_training.id2.to_list()))

In [ ]:
lm2019Dir = workDir + 'srcData/Lugo-Martinez2019/'
lm2019Pairs_trainingFilenames = \
  [lm2019Dir + 'integrated_training_all_model.tsv',
   lm2019Dir + 'integrated_test_all_model.tsv']
lm2019Pairs_training = \
  pd.concat(
      [pd.read_csv(f, sep=' ', dtype='str', header=None, names=['id1', 'id2'])
      for f in lm2019Pairs_trainingFilenames], axis=0, ignore_index=True)

lm2019Pairs_labelFilenames = \
  [lm2019Dir + 'integrated_training_labels_all_model.tsv',
   lm2019Dir + 'integrated_test_labels_all_model.tsv']
lm2019Pairs_label = \
  pd.concat(
      [pd.read_csv(f, sep=' ', dtype='int', header=None, names=['label'])
      for f in lm2019Pairs_labelFilenames], axis=0, ignore_index=True)

lm2019Pairs_training = \
  pd.concat([lm2019Pairs_training, lm2019Pairs_label], axis=1)

lm2019Prots = set(lm2019Pairs_training.id1.to_list()).union(
    set(lm2019Pairs_training.id2.to_list()))

In [ ]:
print('-------------------------------------------------------------------')
print('# hu.MAP 1 proteins: {0} (3803 UniProt)'.format(len(humap1Prots)))
print('# hu.MAP 2 proteins: {0} (2985 UniProt)'.format(len(humap2Prots)))
print('# LM2019 proteins: {0}'.format(len(lm2019Prots)))
print('# CORUM 2018 Release proteins: {0} (3375 UniProt)\n'.format(len(humanProts)))
print('Note: LM2019 proteins are a subset of hu.MAP 1 proteins')
print('--------------------------------\n')

print('# proteins common to hu.MAP 1 and the CORUM 2018 Release: {0}'.format(
    len(humanProts.intersection(humap1Prots))))
print('# proteins common to hu.MAP 1 and the CORUM 2018 Release: {0}'.format(
    len(humanProts.intersection(humap2Prots))))
print('# proteins common to LM2019 and the CORUM 2018 Release: {0}'.format(
    len(humanProts.intersection(lm2019Prots))))
print('# proteins common to hu.MAP 1 and hu.MAP 2: {0}'.format(
    len(humap1Prots.intersection(humap2Prots))))
print('# proteins common to (hu.MAP 1+2) and the CORUM 2018 Release: {0}'.format(
    len(set(list(humap1Prots.union(humap2Prots))).intersection(humanProts))))
print('--------------------------------\n')

print('# proteins across hu.MAP 1, 2, and the CORUM 2018 Release: {0}'.format(
    len(humanProts.union(humap1Prots).union(humap2Prots))))
print('-------------------------------------------------------------------')

-------------------------------------------------------------------
# hu.MAP 1 proteins: 3835 (3803 UniProt)
# hu.MAP 2 proteins: 2993 (2985 UniProt)
# LM2019 proteins: 2163
# CORUM 2018 Release proteins: 3375 (3375 UniProt)

Note: LM2019 proteins are a subset of hu.MAP 1 proteins
--------------------------------

# proteins common to hu.MAP 1 and the CORUM 2018 Release: 2267
# proteins common to hu.MAP 1 and the CORUM 2018 Release: 2791
# proteins common to LM2019 and the CORUM 2018 Release: 2093
# proteins common to hu.MAP 1 and hu.MAP 2: 2141
# proteins common to (hu.MAP 1+2) and the CORUM 2018 Release: 2925
--------------------------------

# proteins across hu.MAP 1, 2, and the CORUM 2018 Release: 5137
-------------------------------------------------------------------


In [ ]:
humap1Pairs = operations.generatePairs(humap1Prots)
humap2Pairs = operations.generatePairs(humap2Prots)
humanPairs = operations.generatePairs(humanProts)

allPairs = set().union(*[humap1Pairs, humap2Pairs, humanPairs])

In [ ]:
humap1Pairs_training = \
  operations.freezePairs(humap1Pairs_training, 'id1', 'id2', pool=True)
humap2Pairs_training = \
  operations.freezePairs(humap2Pairs_training, 'id1', 'id2', pool=True)
lm2019Pairs_training = \
  operations.freezePairs(lm2019Pairs_training, 'id1', 'id2', pool=True)

allPairs_training = \
  set().union(*[set(humap1Pairs_training.pairsFrozen.to_list()),
                set(humap2Pairs_training.pairsFrozen.to_list()),
                set(lm2019Pairs_training.pairsFrozen.to_list()),
                set(humanPairs)])

In [ ]:
print('-----------------------------------------------------------------')
print('# hu.MAP 1 protein-generated pairs: {0}'.format(len(humap1Pairs)))
print('# hu.MAP 2 protein-generated pairs: {0}'.format(len(humap2Pairs)))
print('# CORUM 2018 Release protein-generated pairs: {0}'.format(len(humanPairs)))
print('================================')
print('# hu.MAP 1 pairs: {0}'.format(
    len(set(humap1Pairs_training.pairsFrozen.to_list()))))
print('# hu.MAP 2 pairs: {0}'.format(
    len(set(humap2Pairs_training.pairsFrozen.to_list()))))
print('--------------------------------\n')

print('# pairs common to hu.MAP 1 protein-generated pairs and ' +
      'hu.MAP 1 pairs: {0}'.format(
          len(humap1Pairs.intersection(
              set(humap1Pairs_training.pairsFrozen.to_list())))))
print('# pairs common to hu.MAP 2 protein-generated pairs and ' +
      'hu.MAP 2 pairs: {0}'.format(
          len(humap2Pairs.intersection(
              set(humap2Pairs_training.pairsFrozen.to_list())))))
print('================================')
print('# pairs common to hu.MAP 1 protein-generated pairs and ' +
      'CORUM 2018 Release protein-generated pairs: {0}'.format(
          len(humanPairs.intersection(humap1Pairs))))
print('# pairs common to hu.MAP 2 protein-generated pairs and ' +
      'CORUM 2018 Release protein-generated pairs: {0}'.format(
          len(humanPairs.intersection(humap2Pairs))))
print('================================')
print('# pairs common to hu.MAP 1 pairs and ' +
      'CORUM 2018 Release protein-generated pairs: {0}'.format(
          len(humanPairs.intersection(
              set(humap1Pairs_training.pairsFrozen.to_list())))))
print('# pairs common to hu.MAP 2 pairs and ' +
      'CORUM 2018 Release protein-generated pairs: {0}'.format(
          len(humanPairs.intersection(
              set(humap2Pairs_training.pairsFrozen.to_list())))))
print('--------------------------------\n')

print('# pairs across hu.MAP 1, hu.MAP 2 protein-generated pairs and ' +
      'CORUM 2018 Release protein-generated pairs: {0}'.format(
          len(allPairs)))
print('# pairs across hu.MAP 1, hu.MAP 2 pairs and ' +
      'CORUM 2018 Release protein-generated pairs: {0}'.format(
          len(allPairs_training)))
print('================================')
print('# pairs common to the union of protein-generated pairs and ' +
      'the union of original pairs for sources: \n' +
      'hu.MAP 1, hu.MAP 2, and the CORUM 2018 Release ' +
      'protein-generated pairs: {0}'.format(
          len(allPairs.union(allPairs_training))))
print('================================')

print('# pairs difference between the union of protein-generated pairs and ' +
      'the union of original pairs for sources: \n' +
      'hu.MAP 1, hu.MAP 2, and the CORUM 2018 Release ' +
      'protein-generated pairs: {0}'.format(
          len(allPairs.difference(allPairs_training))))
print('Note: Pairs in the difference have no labels in hu.MAP 1.0 and 2.0 ' +
      'training pairs data; CORUM 2018 pairs do not overlap and CORUM 2012 ' +
      'only overlaps for 20k pairs (in contrast to 1.5M).')
print('-----------------------------------------------------------------')

-----------------------------------------------------------------
# hu.MAP 1 protein-generated pairs: 7351695
# hu.MAP 2 protein-generated pairs: 4477528
# CORUM 2018 Release protein-generated pairs: 5693625
# hu.MAP 1 pairs: 5447238
# hu.MAP 2 pairs: 3435064
--------------------------------

# pairs common to hu.MAP 1 protein-generated pairs and hu.MAP 1 pairs: 5447238
# pairs common to hu.MAP 2 protein-generated pairs and hu.MAP 2 pairs: 3435064
# pairs common to hu.MAP 1 protein-generated pairs and CORUM 2018 Release protein-generated pairs: 2568511
# pairs common to hu.MAP 2 protein-generated pairs and CORUM 2018 Release protein-generated pairs: 3893445
# pairs common to hu.MAP 1 pairs and CORUM 2018 Release protein-generated pairs: 2020627
# pairs common to hu.MAP 2 pairs and CORUM 2018 Release protein-generated pairs: 3031822
--------------------------------

# pairs across hu.MAP 1, hu.MAP 2 protein-generated pairs and CORUM 2018 Release protein-generated pairs: 11043800
# pairs

In [ ]:
allPairs_missingDifference = allPairs.difference(allPairs_training)
print(len(allPairs_missingDifference))
print(allPairs_missingDifference.issubset(humap1Pairs.union(humap2Pairs)))

1529073
True


In [ ]:
trainingPairs_sources = \
  {'allPairs_corumLabeled': allPairs_corumLabeled,
    'humap1Pairs_training': humap1Pairs_training,
    'humap2Pairs_training': humap2Pairs_training,
    'lm2019Pairs_training': lm2019Pairs_training}

In [ ]:
#generates lists of geneID-pair frozensets corresponding to the union and intersection of all pairs respectively
operations.setupSpaces_pairs([v for v in trainingPairs_sources.values()])

###Aggregate labels as strings for duplicate pairs for subsequent analysis

In [ ]:
for k, df in trainingPairs_sources.items():
  print(k)
  df.drop(columns=['id1', 'id2'], inplace=True)
  df.loc[:, 'label'] = df.loc[:, 'label'].astype('str')
  df.rename(columns={'label': k + '_label'}, inplace=True)
  df = df.groupby('pairsFrozen', as_index=False).agg(';'.join)
  trainingPairs_sources[k] = df.copy()

allPairs_corumLabeled
humap1Pairs_training
humap2Pairs_training
lm2019Pairs_training


In [ ]:
#keep count of total unique pairs across
# CORUM (2018), hu.MAP 1, hu.MAP 2., and LM2019
len(set().union(
    *[set(v.pairsFrozen.to_list()) for v in trainingPairs_sources.values()]))

9514727

###Merge training pairs together

In [ ]:
trainingPairs = pd.DataFrame(columns=['pairsFrozen'])
for k, df in trainingPairs_sources.items():
  print(k)
  trainingPairs = trainingPairs.merge(
      df, on=['pairsFrozen'], how='outer')

allPairs_corumLabeled
humap1Pairs_training
humap2Pairs_training
lm2019Pairs_training


In [ ]:
trainingPairs = \
  pd.concat([pd.DataFrame(
      trainingPairs.pairsFrozen.to_list(),
      columns=['id1', 'id2'], dtype='str'),
             trainingPairs], axis=1)

###Identify the source of each pair's label

In [ ]:
trainingPairs.insert(3, 'sourced', np.nan)

trainingPairs.loc[
    trainingPairs.allPairs_corumLabeled_label.notnull(),
    'sourced'] = 'CORUM 2018 Release'
print(trainingPairs.loc[
    trainingPairs.allPairs_corumLabeled_label.notnull()].shape)

trainingPairs.loc[
    (trainingPairs.allPairs_corumLabeled_label.isna() &
     trainingPairs.humap2Pairs_training_label.notnull()),
    'sourced'] = 'hu.MAP 2.0'
print(trainingPairs.loc[
    (trainingPairs.allPairs_corumLabeled_label.isna() &
     trainingPairs.humap2Pairs_training_label.notnull())].shape)

trainingPairs.loc[
    (trainingPairs.allPairs_corumLabeled_label.isna() &
     trainingPairs.humap2Pairs_training_label.isna() &
     trainingPairs.humap1Pairs_training_label.notnull()),
    'sourced'] = 'hu.MAP 1.0'
print(trainingPairs.loc[
    (trainingPairs.allPairs_corumLabeled_label.isna() &
     trainingPairs.humap2Pairs_training_label.isna() &
     trainingPairs.humap1Pairs_training_label.notnull())].shape)

(5693625, 8)
(403242, 8)
(3417860, 8)


In [ ]:
sumTotal = 0
for desc in trainingPairs.sourced.unique():
  sumTotal+=len(trainingPairs.loc[trainingPairs.sourced==desc])
  print('# {0}: {1}'.format(desc, str(sumTotal)))

print('sum total: {0}'.format(sumTotal))

# CORUM 2018 Release: 5693625
# hu.MAP 1.0: 9111485
# hu.MAP 2.0: 9514727
sum total: 9514727


###Identify duplicates

In [ ]:
trainingPairs_dupsCols = \
  [col for col in trainingPairs.columns.to_list()[4:]
   if not all(trainingPairs[col].str.isnumeric())]
print(trainingPairs_dupsCols)

['humap1Pairs_training_label', 'lm2019Pairs_training_label']


In [ ]:
colOrder_pref = \
  ['id1', 'id2', 'pairsFrozen', 'sourced',
   'allPairs_corumLabeled_label',
   'humap1Pairs_training_label',
   'humap1Pairs_training_label_1', 'humap1Pairs_training_label_2',
   'humap2Pairs_training_label',
   'lm2019Pairs_training_label',
   'lm2019Pairs_training_label_1', 'lm2019Pairs_training_label_2']

colOrder_prefSubs = \
  ['pairsFrozen', 'sourced',
   'allPairs_corumLabeled_label', 'humap2Pairs_training_label',
   'humap1Pairs_training_label_1', 'humap1Pairs_training_label_2',
   'lm2019Pairs_training_label_1', 'lm2019Pairs_training_label_2']

sourceCols = \
  ['allPairs_corumLabeled_label',
   'humap1Pairs_training_label',
   'humap1Pairs_training_label_1', 'humap1Pairs_training_label_2',
   'humap2Pairs_training_label',
   'lm2019Pairs_training_label',
   'lm2019Pairs_training_label_1', 'lm2019Pairs_training_label_2']

dupCols_split = \
  ['humap1Pairs_training_label_1', 'humap1Pairs_training_label_2',
   'lm2019Pairs_training_label_1', 'lm2019Pairs_training_label_2']

In [ ]:
for col in trainingPairs_dupsCols:
  trainingPairs[[col+'_1', col+'_2']] = \
    trainingPairs[col].str.split(';', expand=True)

In [ ]:
trainingPairs = trainingPairs[colOrder_pref]
trainingPairs.fillna(value={col: np.nan for col in dupCols_split}, inplace=True)

###Assess and resolve label agreement across hu.MAP 1.0, hu.MAP 2.0, and LM 2019

In [ ]:
labelAgreement = trainingPairs.loc[:, sourceCols].nunique(axis=1, dropna=True)
trainingPairs.insert(4, 'labelAgreement',
  [True if numLabels==1 else False for numLabels in labelAgreement])

print('# pairs w unanimous agreement: {0}'.format(
    trainingPairs.labelAgreement.sum()))
print('# pairs w/o unanimous agreement: {0}'.format(
    trainingPairs.loc[~trainingPairs.labelAgreement].shape))

# pairs w unanimous agreement: 9505728
# pairs w/o unanimous agreement: (8999, 13)


In [ ]:
trainingPairs.insert(4, 'finalLabel', np.nan)

####Final label: value for which all non-empty sources agree

In [ ]:
trainingPairs.loc[trainingPairs.labelAgreement, 'finalLabel'] = \
  trainingPairs.loc[trainingPairs.labelAgreement, sourceCols].fillna(
      method='bfill', axis=1).iloc[:, 0]
print(trainingPairs.loc[trainingPairs.finalLabel.notnull()].shape)

(9505728, 14)


####Final label: CORUM 2018-generated label supercedes hu.MAP 1.0, 2.0 and LM 2019

In [ ]:
trainingPairs.loc[
    ((trainingPairs.finalLabel.isna()) &
     (trainingPairs.allPairs_corumLabeled_label.notnull()) &
     (trainingPairs.sourced=='CORUM 2018 Release')), 'finalLabel'] = \
      trainingPairs.loc[
          ((trainingPairs.finalLabel.isna()) &
           (trainingPairs.allPairs_corumLabeled_label.notnull()) &
            (trainingPairs.sourced=='CORUM 2018 Release')),
          'allPairs_corumLabeled_label']
print(trainingPairs.loc[trainingPairs.finalLabel.notnull()].shape)

(9514258, 14)


####Final label: hu.MAP 2.0 label supercedes hu.MAP 1.0 and LM 2019

In [ ]:
trainingPairs.loc[((trainingPairs.finalLabel.isna()) &
                   (trainingPairs.allPairs_corumLabeled_label.isna()) &
                   (trainingPairs.sourced=='hu.MAP 2.0')), 'finalLabel'] = \
                      trainingPairs.loc[((trainingPairs.finalLabel.isna()) &
                       (trainingPairs.allPairs_corumLabeled_label.isna()) &
                        (trainingPairs.sourced=='hu.MAP 2.0')),
                                        'humap2Pairs_training_label']
print(trainingPairs.loc[trainingPairs.finalLabel.notnull()].shape)

(9514272, 14)


####Final label: Set final label for remaining pairs (multiple records with inconsistent labeling, without agreement with other sources)

In [ ]:
trainingPairs_corum2012_sourced = \
  trainingPairs.loc[((trainingPairs.finalLabel.isna()) &
   (trainingPairs.allPairs_corumLabeled_label.isna()) &
    (trainingPairs.humap2Pairs_training_label.isna()) &
     (trainingPairs.sourced=='hu.MAP 1.0')), 'pairsFrozen'].to_list()
print(len(trainingPairs_corum2012_sourced))

455


In [ ]:
corumData_2012Filename = workDir + 'srcData/labeledPairs_corum2012.tsv'
corumData_2012 = \
  pd.read_csv(corumData_2012Filename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str', 'label': 'int'})
corumData_2012 = operations.freezePairs(corumData_2012, 'id1', 'id2')
corumData_2012Prots = set(corumData_2012.id1.to_list()).union(
    set(corumData_2012.id2.to_list()))

corumData_2012 = \
  corumData_2012.loc[corumData_2012.pairsFrozen.isin(
      trainingPairs_corum2012_sourced)]

In [ ]:
trainingPairs.loc[trainingPairs.pairsFrozen.isin(
    trainingPairs_corum2012_sourced), 'finalLabel'] = \
      trainingPairs.loc[trainingPairs.pairsFrozen.isin(
          trainingPairs_corum2012_sourced), 'pairsFrozen'].map(
              dict(zip(corumData_2012.pairsFrozen, corumData_2012.label)))

In [ ]:
#Only 3 pairs remain for which the proteins constituting the pair are human,
#e.g., the 381 others are either from rat or mouse.  Uniprot.org's ID
#mapping feature in concert with the "IntAct" database confirmed these pairs
#interact
trainingPairs.loc[
    trainingPairs.finalLabel.isna() &
    trainingPairs.id1.isin(set(geneSpace.geneIDs_all)) &
    trainingPairs.id2.isin(set(geneSpace.geneIDs_all)), 'finalLabel'] = 1

In [ ]:
trainingPairs.dropna(subset=['finalLabel'], inplace=True)
trainingPairs.finalLabel = trainingPairs.finalLabel.astype('int')
print(trainingPairs.shape)

(9514346, 14)


In [ ]:
trainingPairs_sourcesFilename = \
  workDir + 'setup/' + \
  'trainingPairs_sourcesLabeled_detailed_corum2018_humap1+2_lm2019.tsv.tar.bz2'
trainingPairs.drop(columns=['pairsFrozen']).to_csv(
    trainingPairs_sourcesFilename, sep='\t', index=False)

trainingPairs_simpleFilename = \
  workDir + 'setup/' + \
  'trainingPairs_labeledAggregate_corum2018_humap1+2_lm2019.tsv.tar.bz2'
trainingPairs.loc[:, ['id1', 'id2', 'finalLabel']].rename(
    columns={'finalLabel': 'label'}).to_csv(
        trainingPairs_simpleFilename, sep='\t', index=False)

##Unlabeled, protein-coding

In [ ]:
nontrainingPairs_proteinCoding = \
  geneIDs_proteinCoding.difference(set(
      trainingPairs.pairsFrozen.to_list()))
print(len(geneIDs_proteinCoding))
print(len(nontrainingPairs_proteinCoding))

pd.DataFrame(nontrainingPairs_proteinCoding,
             columns=['id1', 'id2'], dtype='str').to_csv(
                 workDir + 'setup/nontrainingPairs_proteinCoding.tsv.tar.bz2',
                 sep='\t', index=False)

#Aggregate protein-pair level data
*   hu.MAP 1.0
*   hu.MAP 2.0
*   BioPlex 3.0

##Align and aggregate columns from sources with overlapping features

Rethink:

1) load feat mat(s)

2) copy records for only training pairs

3) account for pairs disjoint between hu.MAP 1.0, 2.0 releases; append columns disjoint between releases for shared pairs and for disjoint pairs, restore/retain hu.MAP 1.0 features

In [ ]:
featMat_humap1Dir = workDir + 'srcData/huMAP1/feature_matrix.txt.gz'
featMat_humap2Dir = workDir + \
  'srcData/huMAP2/humap2_feature_matrix_20200820.featmat.gz'
featMat_bioplex3Dir = workDir + \
  'srcData/BioPlex3/BioPlex_BaitPreyPairs_noFilters_293T_10K_Dec_2019.tsv'

In [ ]:
bioplexFeats_orig = \
  ['nwdscore', 'zscore', 'plate_zscore', 'entropy', 'uPeps', 'ratio',
   'total_psms', 'ratioTotalPSMs', 'UtoTratio']

humanNet_feats = \
  ['CE-CC', 'CE-CX', 'CE-GT', 'CE-LC', 'CE-YH', 'DM-PI', 'HS-CX', 'HS-DC',
   'HS-GN', 'HS-MS', 'HS-PG', 'HS-YH', 'SC-CC', 'SC-CX', 'SC-GT', 'SC-LC',
   'SC-MS', 'SC-TS', 'SC-YH']

colsHumap1_discard = \
  ['enrichment', 'hit',
   'id1', 'id2', 'frozenset_ids',
   'geneid1', 'geneid2', 'frozenset_geneids',
   'gene_id1_x', 'gene_id1_y', 'gene_id1_str_x', 'gene_id1_str_y',
   'gene_id2_x', 'gene_id2_y', 'gene_id2_str_x', 'gene_id2_str_y',
   'frozenset_geneids_x', 'frozenset_geneids_y', 'frozenset_geneids_str_order']

colsHumap1 = list(pd.read_csv(
    featMat_humap1Dir, sep=',', header=None, nrows=1).values[0])
colsHumap1 = [col for col in colsHumap1 if col not in colsHumap1_discard]
print(len(colsHumap1))

258


In [ ]:
colsHumap2_discard = ['id1', 'id2']
colsHumap2 = list(pd.read_csv(
    featMat_humap2Dir, sep=',', header=None, nrows=1).values[0])
colsHumap2 = [col for col in colsHumap2 if col not in colsHumap2_discard]
print(len(colsHumap2))

292


In [ ]:
colsBioplex3_discard = \
  ['CompPASS_ID', 'bait_symbol', 'bait_geneid', 'db_protein_id', 'symbol',
   'prot_description', 'gene_id']
colsBioplex3 = list(pd.read_csv(
    featMat_bioplex3Dir, sep='\t', header=None, nrows=1).values[0])
colsBioplex3 = \
  [col for col in colsBioplex3
   if all([col not in colsBioplex3_discard, isinstance(col, str)])]
print(len(colsBioplex3))

13


In [ ]:
colsHumap1_only = \
  [col for col in colsHumap1 if col not in colsHumap2+bioplexFeats_orig]
print(len(colsHumap1_only))

colsHumap2_only = \
  [col for col in colsHumap2 if col not in colsHumap1]
print(len(colsHumap2_only))

colsHumap1_humap2Order = [col for col in colsHumap2 if col in colsHumap1]
featIdx_lastWan = colsHumap1_humap2Order.index('Sp_iex_60_121019_pq_euc')+1
colsHumap1_humap2Order[featIdx_lastWan: featIdx_lastWan] = humanNet_feats
print(len(colsHumap1_humap2Order))

colsHumap1p2_order = colsHumap1_humap2Order + colsHumap2_only
print(len(colsHumap1p2_order))

19
62
249
311


##Extract desired features

###hu.MAP 1.0 (BioPlex 1.0 included)

Features (258):

*   Wan 2015: 220 (Poisson, Apex, WCC, and PQ-EUC for 55 samples each)
*   Guruharsha 2011: 1 (HGSCore - AP-MS fly feature)
*   Malovannaya 2011: 1 (MEMO-based confidence - AP-MS human feature)
*   HumanNet 2011: 19 (multi-'omics weighted confidence scores)

*   BioPlex 1.0: 9+2
    
    (NWD Score, Z Score, Plate Z Score, Entropy, Unique Peptide Bins, Ratio, Total PSMs, Ratio Total PSMs, and Unique Total Peptide Ratio) + derived MS feats, pair count and -ln(pval)

*   Hein 2015: 4+2

    (prey.bait.correlation, valid.values, log10.prey.bait.ratio, and log10.prey.bait.expression.ratio) + derived MS feats, pair count and -ln(pval)

In [ ]:
#hu.MAP 1.0 labeled pairs, ~400k
featMat_humap1 = \
  pd.read_csv(featMat_humap1Dir, sep=',',
              dtype={'geneid1': 'str', 'geneid2': 'str'}, low_memory=False)
print('# hu.MAP 1.0 dimensions: {0}'.format(featMat_humap1.shape))

featMat_humap1 = \
  operations.freezePairs(featMat_humap1, 'geneid1', 'geneid2', pool=True)
featMat_humap1 = \
  featMat_humap1.loc[featMat_humap1.pairsFrozen.isin(
      trainingPairs.pairsFrozen.to_list()),
                     colsHumap1_only+['pairsFrozen']].copy()
print('# hu.MAP 1.0 dimensions (extraction): {0}'.format(featMat_humap1.shape))

featMat_humap1 = \
  featMat_humap1.groupby('pairsFrozen', as_index=False).mean(numeric_only=True)
print('# hu.MAP 1.0 dimensions (averaged duplicate observations): {0}'.format(
    featMat_humap1.shape))

featMat_humap1Filename = \
  workDir + 'featureData/pairLevel/featMat_trainingPairs_humap1.tsv'
pd.concat(
    [pd.DataFrame(featMat_humap1.pairsFrozen.to_list(), columns=['id1', 'id2'],
                  dtype='str'),
     featMat_humap1.drop(columns=['pairsFrozen'])], axis=1).to_csv(
         featMat_humap1Filename, sep='\t', index=False)

# hu.MAP 1.0 dimensions (extraction): (442755, 20)
# hu.MAP 1.0 dimensions (averaged duplicate observations): (442716, 20)


In [ ]:
#hu.MAP 1.0 unlabeled pairs, ~2M
featMat_humap1Unlabeled = \
  pd.read_csv(featMat_humap1Dir, sep=',',
              dtype={'geneid1': 'str', 'geneid2': 'str'}, low_memory=False)
print('# hu.MAP 1.0 dimensions: {0}'.format(
    featMat_humap1Unlabeled.shape))

featMat_humap1Unlabeled = operations.freezePairs(
    featMat_humap1Unlabeled, 'geneid1', 'geneid2', pool=True)
featMat_humap1Unlabeled = \
  featMat_humap1Unlabeled.loc[featMat_humap1Unlabeled.pairsFrozen.isin(
      nontrainingPairs_proteinCoding)].copy()
print('# hu.MAP unlabeled 1.0 dimensions (extraction): {0}'.format(
    featMat_humap1Unlabeled.shape))

featMat_humap1Unlabeled = featMat_humap1Unlabeled.groupby(
    'pairsFrozen', as_index=False).mean(numeric_only=True)
featMat_humap1Unlabeled = \
  featMat_humap1Unlabeled.loc[:, ['geneid1', 'geneid2'] + humanNet_feats].copy()
featMat_humap1Unlabeled.rename(
    columns={'geneid1': 'id1', 'geneid2': 'id2'}, inplace=True)
print('# hu.MAP unlabeled 1.0 dimensions (extraction): {0}'.format(
    featMat_humap1Unlabeled.shape))

featMat_humap1Unlabeled_filename = \
  workDir + 'featureData/pairLevel/featMat_unlabeledPairs_humap1.tsv'
pd.concat(
    [pd.DataFrame(featMat_humap1Unlabeled.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     featMat_humap1Unlabeled.drop(columns=['pairsFrozen'])], axis=1).to_csv(
         featMat_humap1Unlabeled_filename, sep='\t', index=False)

###hu.MAP 2.0 (BioPlex 1.0 and 2.0 included)

Features (292):

*   BioPlex 2.0: 10+(4=2x2)
    
    (NWD Score, Z Score, Plate Z Score, Entropy, Unique Peptide Bins, Ratio, Total PSMs, Ratio Total PSMs, Unique Total Peptide Ratio and Average Assembled Peptide Spectral Matches) + derived MS feats, pair count and -ln(pval)

*   Gupta 2015: 10=2x5 + 6=3x2

    (Average Spectra, Average Saint probability, Max Saint probability, Fold Change, and Bayesian FDR estimate) + derived feats, pair count and -ln(pval) for both ciliated and non-ciliated data

*   Youn 2018: 5+(6=3x2)

    (Average Spectra, Average Saint probability, Max Saint probability, Fold Change, and Bayesian FDR estimate) + derived MS feats, pair count and -ln(pval)

*   Boldt 2016: 4+(4=2x2)

   (Socioaffinity index SAij, spoke model terms i,j -> bait Sij and Sji, and matrix model for socioaffinity index Mij) + derived feats, pair count and -ln(pval)

*   Treiber: 4=2*2 derived feats, pair count and -ln(pval)

        hu.MAP 2.0 retained 239 features from hu.MAP 1.0, adding 53 additional
        (Wan 2015, Guruharsha 2011, Malovannaya 2011, BioPlex 1.0 w/ derived features, Hein 2015 w/ derived features)

        Note: Derived features, pair count and ln(pval) were calculated by source for multiple thresholds on spectral counts (e.g., all, 2, 4)

In [ ]:
#hu.MAP 2.0 labeled pairs, ~2.3M
featMat_humap2 = \
  pd.read_csv(featMat_humap2Dir, sep=',',
              dtype={'id1': 'str', 'id2': 'str'}, low_memory=False)
print('# hu.MAP 2.0 dimensions: {0}'.format(featMat_humap2.shape))

featMat_humap2 = operations.freezePairs(featMat_humap2, 'id1', 'id2', pool=True)
featMat_humap2 = \
  featMat_humap2.loc[featMat_humap2.pairsFrozen.isin(
      trainingPairs.pairsFrozen.to_list())].copy()
print('# hu.MAP 2.0 dimensions (extraction): {0}'.format(featMat_humap2.shape))

featMat_humap2 = \
  featMat_humap2.groupby('pairsFrozen', as_index=False).mean(numeric_only=True)
print('# hu.MAP 2.0 dimensions (averaged duplicate observations): {0}'.format(
    featMat_humap2.shape))

featMat_humap2Filename = \
  workDir + 'featureData/pairLevel/featMat_trainingPairs_humap2.tsv'
pd.concat(
    [pd.DataFrame(featMat_humap2.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     featMat_humap2.drop(columns=['pairsFrozen'])],
    axis=1).to_csv(featMat_humap2Filename, sep='\t', index=False)

In [ ]:
#hu.MAP 2.0 unlabeled pairs, ~14M
featMat_humap2Unlabeled = \
  pd.read_csv(featMat_humap2Dir, sep=',',
              dtype={'id1': 'str', 'id2': 'str'}, low_memory=False)
print('# hu.MAP 2.0 dimensions: {0}'.format(featMat_humap2Unlabeled.shape))

featMat_humap2Unlabeled = \
  operations.freezePairs(featMat_humap2Unlabeled, 'id1', 'id2', pool=True)
featMat_humap2Unlabeled = \
  featMat_humap2Unlabeled.loc[featMat_humap2Unlabeled.pairsFrozen.isin(
      nontrainingPairs_proteinCoding)].copy()
print('# hu.MAP unlabeled 2.0 dimensions (extraction): {0}'.format(
    featMat_humap2Unlabeled.shape))

featMat_humap2Unlabeled = \
  featMat_humap2Unlabeled.groupby('pairsFrozen', as_index=False).mean(
      numeric_only=True)
print('# hu.MAP unlabeled 2.0 dimensions (averaged duplicate observations):' + \
      '{0}'.format(featMat_humap2Unlabeled.shape))

featMat_humap2Unlabeled_filename = \
  workDir + 'featureData/pairLevel/featMat_unlabeledPairs_humap2.tsv'
pd.concat(
    [pd.DataFrame(featMat_humap2Unlabeled.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     featMat_humap2Unlabeled.drop(columns=['pairsFrozen'])], axis=1).to_csv(
         featMat_humap2Unlabeled_filename, sep='\t', index=False)

###BioPlex 3.0

Features (13):

NWD Score, Z Score, Plate Z Score, Entropy, Unique Peptide Bins, Ratio, Total PSMs, Ratio Total PSMs, Unique Total Peptide Ratio and Average Assembled Peptide Spectral Matches, pInt (probability of interaction), pNI (probability of no interaction), pW (probability wrong)

In [ ]:
#BioPlex 3.0 labeled pairs, ~600k
colOrder_pref = \
  ['bait_geneid', 'gene_id',
   'ave_apsm', 'nwdscore', 'zscore', 'plate_zscore', 'entropy', 'uPeps',
   'ratio', 'total_psms', 'ratioTotalPSMs', 'UtoTratio',
   'pWrongID', 'pNoInt', 'pInt']

featMat_bioplex3 = \
  pd.read_csv(featMat_bioplex3Dir, sep='\t',
              dtype={'bait_geneid': 'str', 'gene_id': 'str'}, low_memory=False)
print('# BioPlex 3.0 dimensions: {0}'.format(featMat_bioplex3.shape))

featMat_bioplex3 = featMat_bioplex3[colOrder_pref].copy()
featMat_bioplex3.rename(
    columns={'bait_geneid': 'id1', 'gene_id': 'id2'}, inplace=True)
featMat_bioplex3.rename(
    columns={col: col+'_bioplex3'
             for col in featMat_bioplex3.columns.to_list()[2:]}, inplace=True)
featMat_bioplex3 = \
  operations.freezePairs(featMat_bioplex3, 'id1', 'id2', pool=True)

featMat_bioplex3 = \
    featMat_bioplex3.loc[featMat_bioplex3.pairsFrozen.isin(
        trainingPairs.pairsFrozen.to_list())].copy()
print('# BioPlex v3.0 dimensions (extraction): {0}'.format(
    featMat_bioplex3.shape))

featMat_bioplex3 = \
  featMat_bioplex3.groupby('pairsFrozen', as_index=False).mean(numeric_only=True)
print('# BioPlex v3.0 dimensions (averaged duplicate observations): {0}'.format(
    featMat_bioplex3.shape))

featMat_bioplex3Filename = \
  workDir + 'featureData/pairLevel/featMat_trainingPairs_bioplex3.tsv'
pd.concat(
    [pd.DataFrame(featMat_bioplex3.pairsFrozen.to_list(), columns=['id1', 'id2'],
                  dtype='str'),
     featMat_bioplex3.drop(
         columns=['pairsFrozen',
                  'pWrongID_bioplex3',
                  'pNoInt_bioplex3',
                  'pInt_bioplex3'])], axis=1).to_csv(
             featMat_bioplex3Filename, sep='\t', index=False)

# BioPlex 3.0 dimensions: (5851900, 21)
# BioPlex v3.0 dimensions (extraction): (614074, 16)
# BioPlex v3.0 dimensions (averaged duplicate observations): (586385, 14)


In [ ]:
#BioPlex 3.0 unlabeled pairs, ~5M
featMat_bioplex3Unlabeled = \
  pd.read_csv(featMat_bioplex3Dir, sep='\t',
              dtype={'bait_geneid': 'str', 'gene_id': 'str'}, low_memory=False)
print('# BioPlex 3.0 dimensions: {0}'.format(featMat_bioplex3Unlabeled.shape))

featMat_bioplex3Unlabeled = featMat_bioplex3Unlabeled[colOrder_pref].copy()
featMat_bioplex3Unlabeled.rename(
    columns={'bait_geneid': 'id1', 'gene_id': 'id2'}, inplace=True)
featMat_bioplex3Unlabeled.rename(
    columns={col: col+'_bioplex3'
             for col in featMat_bioplex3Unlabeled.columns.to_list()[2:]},
    inplace=True)
featMat_bioplex3Unlabeled = \
  operations.freezePairs(featMat_bioplex3Unlabeled, 'id1', 'id2', pool=True)
featMat_bioplex3Unlabeled = \
  featMat_bioplex3Unlabeled.loc[featMat_bioplex3Unlabeled.pairsFrozen.isin(
      nontrainingPairs_proteinCoding)].copy()
print('# BioPlex v3.0 dimensions (protein-coding): {0}'.format(
    featMat_bioplex3Unlabeled.shape))

featMat_bioplex3Unlabeled = \
  featMat_bioplex3Unlabeled.groupby('pairsFrozen', as_index=False).mean(
      numeric_only=True)
print('# BioPlex v3.0 dimensions ' +
      '(protein-coding, averaged duplicate observations): {0}'.format(
    featMat_bioplex3Unlabeled.shape))

featMat_bioplex3Unlabeled_filename = \
  workDir + 'featureData/pairLevel/featMat_unlabeledPairs_bioplex3.tsv'
pd.concat(
    [pd.DataFrame(featMat_bioplex3Unlabeled.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     featMat_bioplex3Unlabeled.drop(
         columns=['pairsFrozen',
                  'pWrongID_bioplex3',
                  'pNoInt_bioplex3',
                  'pInt_bioplex3'])], axis=1).to_csv(
             featMat_bioplex3Unlabeled_filename, sep='\t', index=False)

# BioPlex 3.0 dimensions: (5851900, 21)
# BioPlex v3.0 dimensions (unlabeled): (5088302, 16)
# BioPlex v3.0 dimensions (averaged duplicate observations): (4980729, 14)


##Generate pair-level aggregate data matrix

###Find and assess intersection(s) of labeled features' pairs'

In [ ]:
pairsHumap1 = \
  pd.read_csv(featMat_humap1Dir, sep=',',
              usecols=['geneid1', 'geneid2'], dtype='str')
print('# hu.MAP 1.0 pairs: {0}'.format(pairsHumap1.shape[0]))

pairsHumap1.rename(columns={'geneid1': 'id1', 'geneid2': 'id2'}, inplace=True)
pairsHumap1 = operations.freezePairs(pairsHumap1, 'id1', 'id2', pool=True)
pairsHumap1 = set(
    pairsHumap1.loc[
        pairsHumap1.pairsFrozen.isin(trainingPairs.pairsFrozen.to_list()),
        'pairsFrozen'].to_list())
print('# hu.MAP 1.0 labeled: {0}'.format(len(pairsHumap1)))

# hu.MAP 1.0 pairs: 2557860
# hu.MAP 1.0 labeled: 442716


In [ ]:
pairsHumap2 = \
  pd.read_csv(featMat_humap2Dir, sep=',', usecols=['id1', 'id2'], dtype='str')
print('# hu.MAP 2.0 pairs: {0}'.format(pairsHumap2.shape[0]))

pairsHumap2 = operations.freezePairs(pairsHumap2, 'id1', 'id2', pool=True)
pairsHumap2 = set(
    pairsHumap2.loc[
        pairsHumap2.pairsFrozen.isin(trainingPairs.pairsFrozen.to_list()),
        'pairsFrozen'].to_list())
print('# hu.MAP 2.0 labeled: {0}'.format(len(pairsHumap2)))

# hu.MAP 2.0 pairs: 17564755
# hu.MAP 2.0 labeled: 2299721


The following result means that a clean merge between labeled pairs of the hu.MAP 1.0 and 2.0 releases for HumanNet features of the former and all the remaining of the latter is possible.

In [ ]:
pairsHumap1.issubset(pairsHumap2)

True

In [ ]:
pairsBioplex3 = \
  pd.read_csv(featMat_bioplex3Dir, sep='\t',
              usecols=['bait_geneid', 'gene_id'], dtype='str')
print('# BioPlex 3.0 pairs: {0}'.format(pairsBioplex3.shape[0]))

pairsBioplex3 = \
  operations.freezePairs(pairsBioplex3, 'bait_geneid', 'gene_id', pool=True)
pairsBioplex3 = set(
    pairsBioplex3.loc[
        pairsBioplex3.pairsFrozen.isin(trainingPairs.pairsFrozen.to_list()),
        'pairsFrozen'].to_list())
print('# BioPlex 3.0 labeled pairs: {0}'.format(len(pairsBioplex3)))

# BioPlex 3.0 pairs: 5851900
# BioPlex 3.0 labeled pairs: 586385


In [ ]:
print('# Total labeled pair-level gene pairs: {0}'.format(
    len(set().union(*[pairsHumap1, pairsHumap2, pairsBioplex3]))))

# Total labeled pair-level gene pairs: 2399931


####Merge

In [ ]:
featMats_labeledFilenames = glob.glob(workDir + 'featureData/pairLevel/*training*')
featMats_labeled = []
for name in featMats_labeledFilenames:
    df = pd.read_csv(dfName, sep='\t', dtype={'id1': 'str', 'id2': 'str'})
    df = operations.freezePairs(df, 'id1', 'id2', pool=True)
    featMats_labeled.append(df.drop(columns=['id1', 'id2']))

print('merging aggregate pair-feature file...')
featMats_aggLabeled = \
  reduce(lambda left, right: pd.merge(
      left, right, on=['pairsFrozen'], how='outer'), featMats_labeled)
print(featMats_aggLabeled.shape)
print(featMats_aggLabeled.head())

print('saving aggregate pair-feature file...')
featMats_aggLabeled_filename = \
  workDir + 'featureData/pairLevel/featMats_aggLabeled.tsv.tar.bz2'
pd.concat(
    [pd.DataFrame(featMats_aggLabeled.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     featMats_aggLabeled.drop(columns=['pairsFrozen'])],
    axis=1).to_csv(featMats_aggLabeled_filename, sep='\t', index=False)

after merger, dims should be: (2399931, 323)

###Find and assess intersection(s) of all known, unlabeled pairs

In [ ]:
pairsHumap1 = \
  pd.read_csv(featMat_humap1Dir, sep=',',
              usecols=['geneid1', 'geneid2'], dtype='str')
print('# hu.MAP 1.0 pairs: {0}'.format(pairsHumap1.shape[0]))

pairsHumap1.rename(columns={'geneid1': 'id1', 'geneid2': 'id2'}, inplace=True)
pairsHumap1 = operations.freezePairs(pairsHumap1, 'id1', 'id2', pool=True)
pairsHumap1 = pairsHumap1.loc[(
    pairsHumap1.id1.isin(geneSpace.geneIDs_proteinCoding) &
    pairsHumap1.id2.isin(geneSpace.geneIDs_proteinCoding))].copy()
print('# hu.MAP 1.0 pairs -- protein-coding: {0}'.format(
    pairsHumap1.shape[0]))

pairsHumap1 = \
  set(pairsHumap1.loc[~pairsHumap1.pairsFrozen.isin(
      trainingPairs.pairsFrozen.to_list()), 'pairsFrozen'])
print('# hu.MAP 1.0 pairs -- protein-coding, unlabeled: {0}'.format(len(
    pairsHumap1)))

# hu.MAP 1.0 pairs: 2557860
# hu.MAP 1.0 pairs -- protein-coding: 2524040
# hu.MAP 1.0 pairs -- protein-coding, unlabeled: 2081135


In [ ]:
pairsHumap2 = \
  pd.read_csv(featMat_humap2Dir, sep=',',
              usecols=['id1', 'id2'], dtype='str')
print('# hu.MAP 2.0 pairs: {0}'.format(pairsHumap2.shape[0]))

pairsHumap2 = operations.freezePairs(pairsHumap2, 'id1', 'id2', pool=True)
pairsHumap2 = pairsHumap2.loc[(
    pairsHumap2.id1.isin(geneSpace.geneIDs_proteinCoding) &
    pairsHumap2.id2.isin(geneSpace.geneIDs_proteinCoding))].copy()
#same-protein pairs excluded
pairsHumap2 = pairsHumap2.loc[(pairsHumap2.id1!=pairsHumap2.id2)].copy()
print('# hu.MAP 2.0 pairs -- protein-coding: {0}'.format(pairsHumap2.shape[0]))

pairsHumap2 = \
  set(pairsHumap2.loc[~pairsHumap2.pairsFrozen.isin(
      trainingPairs.pairsFrozen.to_list()), 'pairsFrozen'])
print('# hu.MAP 2.0 pairs -- protein-coding, unlabeled: {0}'.format(len(
    pairsHumap2)))

# hu.MAP 2.0 pairs: 17564755
# hu.MAP 2.0 pairs -- protein-coding: 17263301
# hu.MAP 2.0 pairs -- protein-coding, unlabeled: 14965773


The following result means that a clean merge between unlabeled pairs of the hu.MAP 1.0 and 2.0 releases for HumanNet features of the former and all the remaining of the latter is possible.

In [ ]:
pairsHumap1.issubset(pairsHumap2)

True

In [ ]:
pairsBioplex3 = \
  pd.read_csv(featMat_bioplex3Dir, sep='\t',
              usecols=['bait_geneid', 'gene_id'], dtype='str')
print('# BioPlex 3.0 pairs: {0}'.format(pairsBioplex3.shape[0]))

pairsBioplex3.rename(
    columns={'bait_geneid': 'id1', 'gene_id': 'id2'}, inplace=True)
pairsBioplex3 = operations.freezePairs(pairsBioplex3, 'id1', 'id2', pool=True)
pairsBioplex3 = pairsBioplex3.loc[(
    pairsBioplex3.id1.isin(geneSpace.geneIDs_proteinCoding) &
    pairsBioplex3.id2.isin(geneSpace.geneIDs_proteinCoding))].copy()
print('# BioPlex 3.0 pairs -- protein-coding: {0}'.format(
    pairsBioplex3.shape[0]))

pairsBioplex3 = \
  set(pairsBioplex3.loc[~pairsBioplex3.pairsFrozen.isin(
      trainingPairs.pairsFrozen.to_list()), 'pairsFrozen'])
print('# BioPlex 3.0 pairs -- protein-coding, unlabeled: {0}'.format(len(
    pairsBioplex3)))

# BioPlex 3.0 pairs: 5851900
# BioPlex 3.0 pairs -- protein-coding: 5701981
# BioPlex 3.0 pairs -- protein-coding, unlabeled: 4980729


In [ ]:
print('# Total unlabeled, protein-coding gene pairs: {0}'.format(
    len(set().union(*[pairsHumap1, pairsHumap2, pairsBioplex3]))))

# Total unlabeled, protein-coding gene pairs: 16727935


####Merge

In [ ]:
featMats_unlabeledFilenames = \
  glob.glob(workDir + 'featureData/pairLevel/*unlabeled*')
featMats_unlabeled = []
for dfName in featMats_unlabeledFilenames:
    df = pd.read_csv(dfName, sep='\t', dtype={'id1': 'str', 'id2': 'str'})
    df = operations.freezePairs(df, 'id1', 'id2', pool=True)
    featMats_unlabeled.append(df.drop(columns=['id1', 'id2']))

print('merging aggregate pair-feature file...')
featMats_aggUnlabeled = \
  reduce(lambda left, right: pd.merge(
      left, right, on=['pairsFrozen'], how='outer'), featMats_unlabeled)
print(featMats_aggUnlabeled.shape)
print(featMats_aggUnlabeled.head())

print('saving aggregate pair-feature file...')
featMats_aggUnlabeled_filename = \
  workDir + 'featureData/pairLevel/featMats_aggUnlabeled.tsv.tar.bz2'
pd.concat(
    [pd.DataFrame(featMats_aggUnlabeled.pairsFrozen.to_list(),
                  columns=['id1', 'id2'], dtype='str'),
     featMats_aggUnlabeled.drop(columns=['pairsFrozen'])],
    axis=1).to_csv(featMats_aggUnlabeled_filename, sep='\t', index=False)

after merger, dims should be: (16727935, 323)

#Generate and aggregate protein-level features

*   .....Pair count and ln(pval) for each spectral count thresholds

    *BioPlex 3.0*

*   .....Prey-prey and bait-bait Pearson correlation coefficients

    *BioPlex 3.0*

    *Hein 2015*

*   NCI-60 protein, RNA co-expression
*   .....FANTOM5 Pearson correlation coefficient
*   .....gTEX Pearson correlation coefficient
*   .....Uhlén Pearson correlation coefficient
*   HPA-microscopy localization annotation set features

    a) Overlap b) Equality c) Jaccard similarity d) Inclusion
*   .....HPA-microscopy deep-learning multi-label-location classification set features

    a) Overlap b) Equality c) Jaccard similarity d) Inclusion e) Kullback-Leibler divergence
*   SubCellBarCode

    Pearson correlation coefficient of protein expression for cell lines: A431, H322, HCC827, MCF7, U251

##~~Pair count and ln(pval)

In [ ]:
featMat_bioplex3Derived = reference.deriveFeats('BioPlex3')

##~~Prey-prey and bait-bait Pearson correlation coefficients

In [ ]:
featMat_preyPrey_baitBait = reference.deriveFeats('preyPrey_baitBait')

##NCI-60 rna, protein expression

*   NCI-60 RNA exp: (18,031 choose 2) = 162,549,465
*   NCI-60 Prot cell exp: (6,490 choose 2) = 21,056,805
*   NCI-60 Prot tissues exp: (6,982 choose 2) = 24,370,671
*   NCI-60 Prot exp (cell + tissues):
    
    (6,490 choose 2) + (6,982 choose 2) - (5,932 choose 2) = 27,836,130
    
    (7,540 choose 2) or 28,422,030 - 27,836,130 = 585,900 pairs' data unavailable


*   NCI-60 Exp (RNA + prot):

    (18,031 choose 2) + 27,836,130 - [(6,468 choose 2) + (6,958 choose 2) - (5,915 choose 2)] = 162,758,569
    
    (18,060 choose 2) or 163,072,770 - 162,758,569 = 314,201 pairs' data unavailable

In [ ]:
nci60Feats = reference.nci60Feats(workDir, save=True)
nci60Feats.generate()
nci60Feats.saveMat()

##~~gTEX Pearson correlation coefficient

In [ ]:
featMat_gTEX = reference.deriveFeats('gTEX')

##~~Uhlén Pearson correlation coefficient

In [ ]:
featMat_uhlen = reference.deriveFeats('uhlen')

##~~FANTOM5 Pearson correlation coefficient

In [ ]:
featMat_fantom5 = reference.deriveFeats('fantom5')

##HPA-microscopy localization annotation set features

**Latest (2023.06.19) HPA subcellular locations data meta**

*LM 2019 -> v19.3*

**HPA mat columns**
* Gene
* Gene name
* Reliability
* Main location
* Additional location
* Extracellular location
* *Enhanced*
* *Supported*
* *Approved*
* *Uncertain*
* Single-cell variation intensity
* Single-cell variation spatial
* Cell cycle dependency
* GO id

**Evidenciary categories:**
* Enhanced - The antibody has enhanced validation and there is no contradicting data, such as literature describing experimental evidence for a different location.
* Supported - There is no enhanced validation of the antibody, but the annotated localization is reported in literature.
* Approved - The localization of the protein has not been previously described and was detected by only one antibody without additional antibody validation.
* Uncertain - The antibody-staining pattern contradicts experimental data or expression is not detected at RNA level.

**Subcellular locations ('official') considered:**
* Actin filaments
* Aggresome
* Cell Junctions
* Centriolar satellite
* Centrosome
* Cleavage furrow
* Cytokinetic bridge
* Cytoplasmic bodies
* Cytosol
* Endoplasmic reticulum
* Endosomes
* Focal adhesion sites
* Golgi apparatus
* Intermediate filaments
* Kinetochore
* Lipid droplets
* Lysosomes
* Microtubule ends
* Microtubules
* Midbody
* Midbody ring
* Mitochondria
* Mitotic chromosome
* Mitotic spindle
* Nuclear bodies
* Nuclear membrane
* Nuclear speckles
* Nucleoli
* Nucleoli fibrillar center
* Nucleoli rim
* Nucleoplasm
* Peroxisomes
* Plasma membrane
* Rods & Rings
* Vesicles

**Notes:**

HPA Subcellular Location Protein breakdown ??make sure you can explain the 279 -> 126 pair difference clearly??

*   11,320 Ensembl IDs with either non-null Approved or Supported locations
    
    64,065,540 unique pairs

*   10,824 GeneIDs (all protein-coding) mappings
*   (10,824 choose 2) - ((50 choose 2) - (44 choose 2)) multiple-GeneId mapped Ensembl IDs pairs = 58,573,950 unique pairs

In [ ]:
hpaFeats = reference.hpaFeats(workDir, save=True)
hpaFeats.generate()
hpaFeats.saveMat()

In [ ]:
hpaProts = list(set().union(*
 [df.Protein.to_list() for df in hpaFeats.dataMats.values()]))

In [ ]:
#Submit output to uniprotkb idmapping function
with open('./hpaProteins.txt', 'w') as listFile:
  for p in hpaProts:
    listFile.write(p + '\n')

In [ ]:
proteinLevel_dir = workDir + 'featureData/proteinLevel/'
idMapping_filename = proteinLevel_dir + 'idmapping_hpa.tsv'
hpaProts_geneidMappings = pd.read_csv(idMapping_filename, sep='\t')
hpaProts_geneidMappings.drop(
    columns=['Entry Name', 'Interacts with', 'Organism'], inplace=True)
hpaProts_geneidMappings.rename(
    columns={'From': 'GeneCards', 'Entry': 'UniProtKB'}, inplace=True)
hpaProts_geneidMappings.loc[:, 'GeneID'] = \
  hpaProts_geneidMappings.loc[:, 'GeneID'].str.replace(';', '')
hpaProts_geneidMappings.dropna(subset=['GeneID'], inplace=True)
hpaProts_geneidMappings.drop_duplicates(
    subset=['GeneCards', 'GeneID'], inplace=True)
hpaProts_geneidMappings = \
  {row.GeneCards: row.GeneID for _, row in hpaProts_geneidMappings.iterrows()}

In [ ]:
hpaFeats.featMat.insert(1, 'geneid1',
                         hpaFeats.featMat.id1.map(hpaProts_geneidMappings))
hpaFeats.featMat.insert(3, 'geneid2',
                         hpaFeats.featMat.id2.map(hpaProts_geneidMappings))
hpaFeats.featMat = \
  operations.freezePairs(hpaFeats.featMat, 'geneid1', 'geneid2', pool=True)

hpaFeats.featMat.dropna(subset=['geneid1', 'geneid2'], inplace=True)
hpaFeats.featMat.drop(columns=['id1', 'id2'], inplace=True)
hpaFeats.featMat.rename(
    columns={'geneid1': 'id1', 'geneid2': 'id2'}, inplace=True)
hpaFeats.featMat.to_csv(proteinLevel_dir + 'hpaFeats_geneidMapped.tsv.tar.bz2')

##~~HPA-microscopy deep-learning multi-label-location classification set features

In [ ]:
featMat_ouyang = reference.deriveFeats('ouyang')

##SubCellBarCode

12,418 GeneCards (11,643 GeneID mappings)

In [ ]:
proteinLevel_dir = workDir + 'featureData/proteinLevel/'
scbcFeats = reference.scbcFeats(workDir, save=True)
scbcFeats.generate()
scbcFeats.saveMat()

In [ ]:
scbcProts = list(set().union(*
 [df.Protein.to_list() for df in scbcFeats.dataMats.values()]))

In [ ]:
#Submit output to uniprotkb idmapping function
with open('./scbcProteins.txt', 'w') as listFile:
  for p in scbcProts:
    listFile.write(p + '\n')

In [ ]:
idMapping_filename = proteinLevel_dir + 'idmapping_scbc.tsv'
scbcProts_geneidMappings = pd.read_csv(idMapping_filename, sep='\t')
scbcProts_geneidMappings.drop(
    columns=['Entry Name', 'Interacts with', 'Organism'], inplace=True)
scbcProts_geneidMappings.rename(
    columns={'From': 'GeneCards', 'Entry': 'UniProtKB'}, inplace=True)
scbcProts_geneidMappings.loc[:, 'GeneID'] = \
  scbcProts_geneidMappings.loc[:, 'GeneID'].str.replace(';', '')
scbcProts_geneidMappings.dropna(subset=['GeneID'], inplace=True)
scbcProts_geneidMappings.drop_duplicates(
    subset=['GeneCards', 'GeneID'], inplace=True)
scbcProts_geneidMappings = \
  {row.GeneCards: row.GeneID for _, row in scbcProts_geneidMappings.iterrows()}

In [ ]:
scbcFeats.featMat.insert(1, 'geneid1',
                         scbcFeats.featMat.id1.map(scbcProts_geneidMappings))
scbcFeats.featMat.insert(3, 'geneid2',
                         scbcFeats.featMat.id2.map(scbcProts_geneidMappings))
scbcFeats.featMat = \
  operations.freezePairs(scbcFeats.featMat, 'geneid1', 'geneid2', pool=True)

scbcFeats.featMat.dropna(subset=['geneid1', 'geneid2'], inplace=True)
scbcFeats.featMat.drop(columns=['id1', 'id2'], inplace=True)
scbcFeats.featMat.rename(
    columns={'geneid1': 'id1', 'geneid2': 'id2'}, inplace=True)
scbcFeats.featMat.to_csv(
    proteinLevel_dir + 'scbcFeats_geneidMapped.tsv.tar.bz2', sep='\t')

##Generate protein-level aggregate data matrix

In [ ]:
featMats_proteinLevel = \
  [featMat_bioplex3Derived, featMat_preyPrey_baitBait, nci60Corr_rnaExp_protExp,
   featMat_gTEX, featMat_uhlen, featMat_fantom5, hpaFeats, featMat_ouyang,
   scbcData]

featMat_proteinLevel_agg = \
  reduce(lambda  left, right: pd.merge(left, right,
                                       on=['pairsFrozen'], how='outer'),
         featMats_proteinLevel)

pickle.dump(featMat_proteinLevel_agg,
            open(workDir + 'featureData/proteinLevel/featMat_trainingPairs_srcAgg.pkl', 'wb'))

#Combine protein-pair-, and protein-level feature matrices

In [ ]:
featMat_agg = \
  featMat_pairLevel_agg.merge(featMat_proteinLevel_agg, on=['pairsFrozen'], how='outer')
pickle.dump(featMat_agg, open(workDir + 'featureData/featMat_agg.pkl', 'wb'))

#Outputs

##Pairs' sets

In [ ]:
#all possible CORUM-labeled pairs
corumLabeled_pairsFilename = setupDir + 'allPairs_corumLabeled.tsv.tar.bz2'
allPairs_corumLabeled = pd.read_csv(corumLabeled_pairsFilename, sep='\t',
                                    dtype={'id1': 'str', 'id2': 'str'})
allPairs_corumLabeled = \
  operations.freezePairs(allPairs_corumLabeled, 'id1', 'id2', pool=True)

In [ ]:
#all possible pairs from protein-coding genes (Entrez Gene) (~213M)
proteinCoding_genesFilename = \
  setupDir + 'allPairs_proteinCoding_geneidFmt.tsv.tar.bz2'
geneIDs_proteinCoding = pd.read_csv(proteinCoding_genesFilename, sep='\t',
                                    dtype={'id1': 'str', 'id2': 'str'})
geneIDs_proteinCoding = \
  operations.freezePairs(geneIDs_proteinCoding, 'id1', 'id2', pool=True)

In [ ]:
#trainingPairs_detailed (~9M)
trainingPairs_filename = setupDir + \
  'trainingPairs_sourcesLabeled_detailed_corum2018_humap1+2_lm2019.tsv.tar.bz2'
trainingPairs = pd.read_csv(trainingPairs_filename, sep='\t',
                            dtype={'id1': 'str', 'id2': 'str'})
trainingPairs = operations.freezePairs(trainingPairs, 'id1', 'id2', pool=True)

In [ ]:
#trainingPairs_simple (~9M)
trainingPairs_filename = setupDir + \
  'trainingPairs_labeledAggregate_corum2018_humap1+2_lm2019.tsv.tar.bz2'
trainingPairs = pd.read_csv(trainingPairs_filename, sep='\t',
                            dtype={'id1': 'str', 'id2': 'str'})
trainingPairs = operations.freezePairs(trainingPairs, 'id1', 'id2', pool=True)

In [ ]:
#unlabeled, all-known protein-coding gene pairs (~206M)
nontrainingPairs_proteinCoding_filename = \
  workDir + 'setup/nontrainingPairs_proteinCoding.tsv.tar.bz2'
nontrainingPairs_proteinCoding = \
  pd.read_csv(nontrainingPairs_proteinCoding_filename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str'})
nontrainingPairs_proteinCoding = \
  operations.freezePairs(nontrainingPairs_proteinCoding, 'id1', 'id2', pool=True)

213077046
206875481


##Pair-level feature matrices

In [ ]:
#(~400k)
featMat_humap1Filename = \
  workDir + 'featureData/pairLevel/featMat_trainingPairs_humap1.tsv.tar.bz2'
featMat_humap1 = pd.read_csv(featMat_humap1Filename, sep='\t',
                             dtype={'id1': 'str', 'id2': 'str'})
featMat_humap1 = operations.freezePairs(featMat_humap1, 'id1', 'id2', pool=True)

In [ ]:
#(~2.3M)
featMat_humap2Filename = \
  workDir + 'featureData/pairLevel/featMat_trainingPairs_humap2.tsv.tar.bz2'
featMat_humap2 = pd.read_csv(featMat_humap2Filename, sep='\t',
                             dtype={'id1': 'str', 'id2': 'str'})
featMat_humap2 = operations.freezePairs(featMat_humap2, 'id1', 'id2', pool=True)

In [ ]:
#(~600k)
featMat_bioplex3Filename = \
  workDir + 'featureData/pairLevel/featMat_trainingPairs_bioplex3.tsv.tar.bz2'
featMat_bioplex3 = pd.read_csv(featMat_bioplex3Filename, sep='\t',
                             dtype={'id1': 'str', 'id2': 'str'})
featMat_bioplex3 = \
  operations.freezePairs(featMat_bioplex3, 'id1', 'id2', pool=True)

In [ ]:
#(~2.3M)
featMat_pairLevel_aggFilename = \
  workDir + 'featureData/pairLevel/featMats_aggLabeled.tsv.tar.bz2'
featMat_pairLevel_agg = \
  pd.read_csv(featMat_pairLevel_aggFilename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str'})

In [ ]:
#(~2M)
featMat_humap1Unlabeled_filename = \
  workDir + 'featureData/pairLevel/featMat_unlabeledPairs_humap1.tsv.tar.bz2'
featMat_humap1Unlabeled = \
  pd.read_csv(featMat_humap1Unlabeled_filename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str'})
featMat_humap1Unlabeled = \
  operations.freezePairs(featMat_humap1Unlabeled, 'id1', 'id2', pool=True)

In [ ]:
#(~14M)
featMat_humap2Unlabeled_filename = \
  workDir + 'featureData/pairLevel/featMat_unlabeledPairs_humap2.tsv.tar.bz2'
featMat_humap2Unlabeled = \
  pd.read_csv(featMat_humap2Unlabeled_filename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str'})
featMat_humap2Unlabeled = \
  operations.freezePairs(featMat_humap2Unlabeled, 'id1', 'id2', pool=True)

In [ ]:
#(~5M)
featMat_bioplex3Unlabeled_filename = \
  workDir + 'featureData/pairLevel/featMat_unlabeledPairs_bioplex3.tsv.tar.bz2'
featMat_bioplex3Unlabeled = \
  pd.read_csv(featMat_bioplex3Unlabeled_filename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str'})
featMat_bioplex3Unlabeled = \
  operations.freezePairs(featMat_bioplex3Unlabeled, 'id1', 'id2', pool=True)

In [ ]:
#(~17M)
featMat_pairLevel_aggUnlabeled_filename = \
  workDir + 'featureData/pairLevel/featMats_aggUnlabeled.tsv.tar.bz2'
featMat_pairLevel_aggUnlabeled = \
  pd.read_csv(featMat_pairLevel_aggUnlabeled_filename, sep='\t',
              dtype={'id1': 'str', 'id2': 'str'})
featMat_pairLevel_aggUnlabeled = \
  operations.freezePairs(featMat_pairLevel_aggUnlabeled, 'id1', 'id2', pool=True)